# CLEAN COLAB NOTEBOOK FOR MD&A EXTRACTION FROM EDGAR 10-K FILINGS

**Purpose:** Extract Management's Discussion and Analysis (MD&A) sections from 10-K filings

**Prerequisites:**
- Repository already cloned to: `/content/drive/MyDrive/EDGAR_Project/edgar-crawler`
- Raw 10-K files downloaded and stored in Google Drive

**Instructions:**
- 🟢 **GREEN cells** = Run EVERY TIME (including resume)
- 🟡 **YELLOW cells** = Run FIRST TIME ONLY (has skip logic)
- 🔵 **BLUE cells** = Run ONLY WHEN NEEDED (optional/conditional)

---

# SECTION 1: SETUP

These cells prepare your Colab environment and connect to Google Drive.

In [ ]:
## 🟢 Cell 1: Mount Google Drive
## RUN: EVERY TIME (first step for any session)
##
## What it does:
## - Connects Colab to your Google Drive
## - Allows access to repository and data files
## - Checks if already mounted to avoid duplicate mount attempts
##
## Expected output: "✅ Drive mounted successfully" or "✅ Drive already mounted"

import os
from google.colab import drive

# Check if already mounted
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Drive already mounted")
else:
    # Mount for first time
    drive.mount('/content/drive')
    print("✅ Drive mounted successfully")

In [ ]:
## 🟢 Cell 2: Navigate to Repository
## RUN: EVERY TIME
##
## What it does:
## - Changes working directory to your cloned repository
## - All subsequent commands run from this directory
## - Verifies you're in the correct location
##
## Expected output: /content/drive/MyDrive/EDGAR_Project/edgar-crawler
##
## NOTE: Repository should already be cloned here. If not, you need to clone it first!

import os

REPO_DIR = '/content/drive/MyDrive/EDGAR_Project/edgar-crawler'

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f"✅ Working directory: {os.getcwd()}")
else:
    print(f"❌ Repository not found at: {REPO_DIR}")
    print("Please clone the repository first!")

In [ ]:
## 🟢 Cell 3: Install Dependencies
## RUN: EVERY TIME (if runtime was restarted)
## SKIP: If runtime is still active and you just disconnected from Drive
##
## What it does:
## - Installs Python packages needed for extraction
## - Uses specific versions to avoid dependency conflicts
## - Includes pyarrow for Parquet file creation
##
## How to know if you need to run:
## - If you see "Runtime disconnected" or "Session crashed" → RUN THIS
## - If you only see "Drive disconnected" → SKIP THIS (dependencies still there)
##
## Expected time: ~30-60 seconds

print("📦 Installing dependencies...")

# Install compatible versions to avoid conflicts
!pip install -q 'dill<0.3.9' 'multiprocess<0.70.17'
!pip install -q pox ppft
!pip install -q --no-deps pathos  # No-deps avoids conflicts
!pip install -q beautifulsoup4 lxml requests pandas tqdm click cssutils numpy
!pip install -q pyarrow  # For Parquet file creation

print("✅ All dependencies installed")

In [ ]:
## 🟢 Cell 4: Activate Keep-Alive Script
## RUN: EVERY TIME (recommended)
##
## What it does:
## - Simulates browser activity to prevent Colab from disconnecting
## - Clicks the connect button every 60 seconds
## - Does NOT guarantee no disconnection (but significantly helps!)
##
## Benefits:
## - Reduces disconnections during long extractions
## - Keeps session alive even if you switch browser tabs
##
## Expected output: "✅ Keep-alive activated"

from IPython.display import display, Javascript

display(Javascript('''
function KeepClicking(){
    console.log("Keeping session alive...");
    document.querySelector("colab-connect-button").click();
}
setInterval(KeepClicking, 60000);  // Click every 60 seconds
'''))

print("✅ Keep-alive activated")
print("💡 This helps prevent disconnection during long extractions")

# SECTION 2: REBUILD METADATA (ONE-TIME SETUP)

**Run this section ONLY on first extraction.**

These cells create a metadata CSV file by scanning all downloaded 10-K files on disk.
Once created, you can skip this section for all future extractions.

In [ ]:
## 🟡 Cell 5: Rebuild Metadata from Files on Disk
## RUN: FIRST TIME ONLY
## SKIP: If datasets/FILINGS_METADATA.csv already exists
##
## What it does:
## - Scans all downloaded 10-K files in RAW_FILINGS/10-K/
## - Extracts CIK, year, accession number from filenames
## - Creates FILINGS_METADATA.csv with all required columns
## - This CSV tells the extractor which files to process
##
## Expected time: 5-10 minutes for ~80,000 files
##
## When to re-run:
## - First time ever
## - If you downloaded NEW 10-K files
## - If FILINGS_METADATA.csv is missing or corrupted

import os
import pandas as pd

METADATA_FILE = 'datasets/FILINGS_METADATA.csv'

# Check if metadata already exists
if os.path.exists(METADATA_FILE):
    metadata = pd.read_csv(METADATA_FILE)
    print(f"✅ Metadata already exists: {len(metadata):,} filings")
    print(f"   To rebuild, delete {METADATA_FILE} and re-run this cell")
else:
    print("📊 Building metadata from files on disk...")
    print("   This may take 5-10 minutes for large datasets\n")
    
    from rebuild_metadata_colab import rebuild_for_colab
    
    # Fast mode: extracts CIK, Type, year, accession_number from filenames
    rebuild_for_colab(filing_types=['10-K'], fast_mode=True, dry_run=False)
    
    # Add required columns for extraction
    metadata = pd.read_csv(METADATA_FILE)
    
    # Add all missing columns with placeholder data
    required_columns = {
        'Company': lambda: 'Company_' + metadata['CIK'].astype(str),
        'Date': lambda: metadata['year'].astype(str) + '-01-01',
        'filing_date': lambda: metadata['year'].astype(str) + '-01-01',
        'Period of Report': lambda: metadata['year'].astype(str) + '-12-31',
        'SIC': lambda: 'Unknown',
        'State of Inc': lambda: 'Unknown',
        'State location': lambda: 'Unknown',
        'Fiscal Year End': lambda: '1231',
        'html_index': lambda: 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + metadata['CIK'].astype(str),
        'complete_text_file_link': lambda: 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + metadata['CIK'].astype(str),
        'htm_file_link': lambda: 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + metadata['CIK'].astype(str),
    }
    
    for col_name, col_value_func in required_columns.items():
        if col_name not in metadata.columns:
            metadata[col_name] = col_value_func()
    
    metadata.to_csv(METADATA_FILE, index=False)
    
    print(f"\n✅ Metadata complete: {len(metadata):,} filings ready for extraction")
    print(f"   Unique companies: {metadata['CIK'].nunique():,}")
    print(f"   Years: {sorted(metadata['year'].unique())}")

# SECTION 3: CONFIGURE EXTRACTION

These cells configure the extraction settings. Run once, then can skip on resume.

In [ ]:
## 🟡 Cell 6: Update Config for MD&A Extraction
## RUN: FIRST TIME ONLY
## SKIP: On resume (config already set)
##
## What it does:
## - Updates config.json to point to your metadata file
## - Sets filing type to 10-K only
## - Ensures extractor knows where to find filing information
##
## Expected output: "✅ Config updated for MD&A extraction"
##
## When to re-run:
## - First time
## - If you pulled new code from repository that changed config.json

import json

config_path = 'config.json'

with open(config_path, 'r') as f:
    config = json.load(f)

config['extract_items']['filings_metadata_file'] = 'FILINGS_METADATA.csv'
config['extract_items']['filing_types'] = ['10-K']

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Config updated for MD&A extraction")

In [ ]:
## 🟡 Cell 7: Verify Subdirectory Support
## RUN: FIRST TIME ONLY (automatic check)
## SKIP: On resume
##
## What it does:
## - Checks if extract_items.py supports year-based subdirectories
## - Main branch should already have this fix
## - If not found, displays a warning (but likely won't be needed)
##
## Background:
## - Some 10-K files are organized in year folders (e.g., 2007/, 2008/)
## - This check ensures the extractor can find files in subdirectories
##
## Expected output: "✅ extract_items.py supports nested directories"

file_path = 'extract_items.py'

with open(file_path, 'r') as f:
    content = f.read()

# Check if already supports subdirectories
if 'os.walk(type_dir)' in content:
    print("✅ extract_items.py supports nested directories")
    print("   Can find files in year-based subdirectories (e.g., 2007/, 2008/)")
else:
    print("⚠️  extract_items.py may not support nested directories")
    print("   Main branch should have this fix - try pulling latest code")
    print("   If extraction fails with 'FileNotFoundError', update from main branch")

# SECTION 4: EXTRACT MD&A

**Main extraction cell - Run every time you want to extract or resume.**

In [ ]:
## 🟢 Cell 8: Run MD&A Extraction
## RUN: EVERY TIME (for first run AND resume)
##
## What it does:
## - Extracts Item 7 (MD&A section) from all 10-K filings
## - Automatically SKIPS already-extracted files (resume-friendly!)
## - Saves results to datasets/EXTRACTED_FILINGS/10-K/ as JSON files
##
## Expected behavior:
## - First run: Starts from file 1, extracts all
## - Resume: Quickly skips already-done files, continues from where it stopped
## - Progress bar shows: X/83628 files processed
## - Speed: ~10-18 files/second (varies by file size)
##
## How resume works:
## - "Skip Existing: True" is set in extraction_configs/mda_only.json
## - Script checks if output JSON exists before processing
## - Skipped files process in ~0.01 seconds (very fast)
## - New extractions take ~5 seconds each (slower)
##
## Expected time:
## - First full run: ~12-24 hours for 83,628 files
## - Resume after 50% done: ~6-12 hours
##
## If disconnected:
## 1. Re-run Cells 1-4 (Mount, Navigate, Dependencies, Keep-alive)
## 2. Re-run this cell - it will resume automatically!
##
## Expected output:
## - Progress bar moving (even if starting at 0%)
## - "Could not extract" messages (normal - some files lack MD&A)
## - Speed indicator (e.g., 17.93it/s)

print("🚀 Starting MD&A extraction...")
print("   This will automatically skip already-extracted files")
print("   Safe to resume after disconnection!\n")

!python flexible_extractor.py --config extraction_configs/mda_only.json

# SECTION 5: CHECK PROGRESS

**Use these cells to monitor extraction progress.**

**⚠️ IMPORTANT:** You can ONLY run these cells when Cell 8 (extraction) is NOT running!
Colab can only run one cell at a time.

In [ ]:
## 🔵 Cell 9: Check Extraction Progress
## RUN: ONLY when extraction cell (Cell 8) is stopped/completed
##
## What it does:
## - Counts how many JSON files have been extracted
## - Compares against total expected files from metadata
## - Shows percentage complete and files remaining
## - Checks sample files for MD&A quality
##
## When to run:
## - BEFORE starting extraction (to see baseline)
## - AFTER extraction completes
## - AFTER a disconnection (to see how much was done)
##
## ⚠️ DO NOT run while Cell 8 is running! (Colab limitation)
##
## Expected output:
## - Extracted: X files
## - Expected: 83,628 files (or your total)
## - Progress: X%
## - Sample quality check showing 3 random files

import os
import json
import pandas as pd

extracted_dir = 'datasets/EXTRACTED_FILINGS/10-K'

if os.path.exists(extracted_dir):
    # Count all JSON files (including in subdirectories if any)
    all_files = []
    for root, dirs, files in os.walk(extracted_dir):
        all_files.extend([os.path.join(root, f) for f in files if f.endswith('.json')])
    
    # Get expected total from metadata
    metadata = pd.read_csv('datasets/FILINGS_METADATA.csv')
    expected = len(metadata[metadata['Type'] == '10-K'])
    
    print(f"📊 Extraction Progress:")
    print(f"   Extracted: {len(all_files):,} files")
    print(f"   Expected: {expected:,} files")
    print(f"   Progress: {len(all_files)/expected*100:.1f}%")
    print(f"   Remaining: {expected - len(all_files):,} files")
    
    # Check sample files for MD&A content
    if len(all_files) > 0:
        print(f"\n📋 Sample Quality Check (first 3 files):")
        for fpath in all_files[:3]:
            fname = os.path.basename(fpath)
            try:
                with open(fpath, 'r') as f:
                    data = json.load(f)
                    has_mda = 'item_7' in data and len(data.get('item_7', '')) > 100
                    mda_len = len(data.get('item_7', ''))
                    print(f"   {fname}: {'✅' if has_mda else '❌'} MD&A ({mda_len:,} chars)")
            except Exception as e:
                print(f"   {fname}: ⚠️ Error reading file - {e}")
else:
    print("❌ No extraction directory found")
    print(f"   Expected location: {extracted_dir}")
    print("   Run Cell 8 to start extraction")

# SECTION 6: CREATE ANALYSIS FILES

**Run this AFTER extraction is complete to create analysis-ready files.**

In [ ]:
## 🔵 Cell 10: Create Analysis Files (CSV + Parquet)
## RUN: AFTER extraction is complete (or mostly complete)
##
## What it does:
## - Reads all extracted JSON files
## - Creates TWO output files:
##   1. mda_metadata.csv (lightweight, just metadata + stats)
##   2. mda_full.parquet (full MD&A text, compressed, ~500MB-1GB)
##
## Output files:
## - mda_metadata.csv: Filename, CIK, company, dates, MD&A length, etc.
## - mda_full.parquet: Full MD&A text + metadata (for text analysis)
##
## Where saved:
## - /content/drive/MyDrive/EDGAR_Project/ (your project root)
##
## When to run:
## - After all extraction is done
## - Can run on partial extractions (will process whatever exists)
##
## Expected time: 5-15 minutes for 80,000+ files
##
## Uses:
## - CSV: Quick viewing in Excel/Google Sheets
## - Parquet: Fast loading in Python/R for text analysis

import pandas as pd
import json
import os
from tqdm import tqdm

print("📊 Creating analysis files...")
print("   This may take 5-15 minutes depending on file count\n")

extracted_dir = 'datasets/EXTRACTED_FILINGS/10-K'
metadata_records = []
full_data_records = []

# Process all extracted JSON files
json_files = []
for root, dirs, files in os.walk(extracted_dir):
    json_files.extend([os.path.join(root, f) for f in files if f.endswith('.json')])

for filepath in tqdm(json_files, desc="Processing"):
    filename = os.path.basename(filepath)
    
    try:
        with open(filepath, 'r') as f:
            filing = json.load(f)
        
        # Metadata (lightweight)
        metadata_records.append({
            'filename': filename,
            'cik': filing.get('cik', ''),
            'company': filing.get('company', ''),
            'filing_date': filing.get('filing_date', ''),
            'period_of_report': filing.get('period_of_report', ''),
            'year': filing.get('period_of_report', '')[:4] if filing.get('period_of_report', '') else '',
            'has_mda': 'item_7' in filing and len(filing.get('item_7', '')) > 0,
            'mda_length': len(filing.get('item_7', '')),
            'json_path': filepath
        })
        
        # Full data (with MD&A text)
        if 'item_7' in filing and filing['item_7']:
            full_data_records.append({
                'cik': filing.get('cik', ''),
                'company': filing.get('company', ''),
                'filing_date': filing.get('filing_date', ''),
                'period_of_report': filing.get('period_of_report', ''),
                'year': filing.get('period_of_report', '')[:4] if filing.get('period_of_report', '') else '',
                'mda_text': filing.get('item_7', '')
            })
    except Exception as e:
        print(f"⚠️  Error processing {filename}: {e}")

# Save files
df_meta = pd.DataFrame(metadata_records)
df_full = pd.DataFrame(full_data_records)

meta_path = '/content/drive/MyDrive/EDGAR_Project/mda_metadata.csv'
parquet_path = '/content/drive/MyDrive/EDGAR_Project/mda_full.parquet'

df_meta.to_csv(meta_path, index=False)
df_full.to_parquet(parquet_path, compression='gzip', index=False)

print(f"\n✅ Files created:")
print(f"   📄 Metadata CSV: {len(df_meta):,} records ({os.path.getsize(meta_path)/1024:.1f} KB)")
print(f"   📦 Parquet: {len(df_full):,} records ({os.path.getsize(parquet_path)/(1024**2):.1f} MB)")
print(f"\n📊 Statistics:")
print(f"   Files with MD&A: {df_meta['has_mda'].sum():,}")
print(f"   Files without MD&A: {(~df_meta['has_mda']).sum():,}")
if len(df_full) > 0:
    print(f"   Years covered: {sorted(df_full['year'].unique())}")
    print(f"   Avg MD&A length: {df_full['mda_text'].str.len().mean():,.0f} chars")
print(f"\n🎉 Ready for analysis!")

# SECTION 7: QUICK RESUME (AFTER DISCONNECTION)

**If Colab disconnects, run ONLY these cells to resume:**

1. Cell 1 (Mount Drive)
2. Cell 2 (Navigate to repo)
3. Cell 3 (Install dependencies) - ONLY if runtime restarted
4. Cell 4 (Keep-alive)
5. Cell 8 (Resume extraction)

**Skip Cells 5-7** (metadata rebuild and config - already done!)

In [ ]:
## 🔵 Cell 11: Quick Resume Helper
## RUN: OPTIONAL - Only if you want a one-click resume
##
## What it does:
## - Combines all resume steps into one cell
## - Checks Drive connection
## - Navigates to repo
## - Resumes extraction
##
## When to use:
## - After a disconnection
## - If you already ran dependencies (Cell 3) and they're still installed
##
## ⚠️ If this fails with import errors, run Cell 3 first!

import os

# Check Drive
if not os.path.exists('/content/drive/MyDrive'):
    print("❌ Drive not mounted! Run Cell 1 first.")
else:
    # Navigate to repo
    os.chdir('/content/drive/MyDrive/EDGAR_Project/edgar-crawler')
    print(f"✅ Ready to resume from: {os.getcwd()}")
    print("\n🔄 Resuming extraction...\n")
    
    # Resume extraction
    !python flexible_extractor.py --config extraction_configs/mda_only.json